In [ ]:
# Alles runnen totdat je CODE MEREL ziet, dan het blokje runnen met jouw naam!

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os

from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    Samplers,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

# Define the folder path and get list of files
folder_name = "selected_4_policies"
file_list = os.listdir(folder_name)

# Initialize empty lists to hold dataframes for policies_20 and policies_21
policies_20_dfs = []
policies_21_dfs = []

for file in file_list:
    if file.endswith('.csv'):
        # Read the CSV
        df = pd.read_csv(os.path.join(folder_name, file), nrows=1)

        # Check if file name contains "_20_" or "_21_" and add sliced df accordingly
        if '_20_' in file:
            # Take rows 20 to 50 by index (i.e. iloc[19:50])
            policies_20_dfs.append(df)
        elif '_21_' in file:
            policies_21_dfs.append(df)


policies_20_df = pd.concat(policies_20_dfs, ignore_index=True).iloc[:, 19:50]
policies_21_df = pd.concat(policies_21_dfs, ignore_index=True).iloc[:, 19:50]

In [ ]:
from ema_workbench import Policy

policies_20 = []

for idx, row in policies_20_df.iterrows():
    # Convert the row (pandas Series) to a dictionary: lever_name -> lever_value
    lever_values = row.to_dict()

    # Create a unique policy name, e.g. "policy_20_row_0"
    policy_name = f"policy_20_row_{idx}"

    # Create the Policy object
    policy = Policy(policy_name, **lever_values)

    # Append to the list
    policies_20.append(policy)

policies_21 = []

for idx, row in policies_21_df.iterrows():
    # Convert the row (pandas Series) to a dictionary: lever_name -> lever_value
    lever_values = row.to_dict()

    # Create a unique policy name, e.g. "policy_20_row_0"
    policy_name = f"policy_21_row_{idx}"

    # Create the Policy object
    policy = Policy(policy_name, **lever_values)

    # Append to the list
    policies_21.append(policy)



In [ ]:
import os
from datetime import datetime
import pandas as pd

def save_experiments_and_outcomes(name, experiments, outcomes, folder_name="robustness_results_other_aggregation"):
    """
    Saves the experiments and outcomes to CSV files in the specified folder,
    with filenames that include a custom name and timestamp.

    Parameters:
    - name (str): Custom label for the output files (e.g., 'policy6_seed20').
    - experiments (pd.DataFrame): The experiments DataFrame.
    - outcomes (dict): The outcomes dictionary from the EMA Workbench.
    - folder_name (str): The name of the folder where files will be saved. Default is "results".
    """
    # Get timestamp
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Ensure folder exists
    os.makedirs(folder_name, exist_ok=True)

    # Define file paths
    exp_path = os.path.join(folder_name, f'{name}_robustness_50000_experiments_{current_date}.csv')
    out_path = os.path.join(folder_name, f'{name}_robustness_50000_outcomes_{current_date}.csv')

    # Save experiments
    experiments.to_csv(exp_path, index=False)

    # Flatten outcomes and save
    flattened_data = {key: value.flatten() for key, value in outcomes.items()}
    df_outcomes = pd.DataFrame(flattened_data)
    df_outcomes.to_csv(out_path, index=False)

    print(f"Saved experiments and outcomes as '{name}' in '{folder_name}' (timestamp: {current_date})")


# CODE MEREL

In [ ]:
n_scenarios = 50000
np.random.seed(21)
policy = policies_20[0]

# Load your model and define parameters
dike_model, planning_steps = get_model_for_problem_formulation(3)

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(
        scenarios=n_scenarios,
        policies=policy,
        uncertainty_sampling=Samplers.LHS
    )

print(results)

experiments, outcomes = results
save_experiments_and_outcomes('policy_6_s20', experiments, outcomes)


# CODE TWAN

In [ ]:
n_scenarios = 50000
np.random.seed(21)
policy = policies_20[1]

# Load your model and define parameters
dike_model, planning_steps = get_model_for_problem_formulation(3)

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(
        scenarios=n_scenarios,
        policies=policy,
        uncertainty_sampling=Samplers.LHS
    )

experiments, outcomes = results
save_experiments_and_outcomes('policy_8_s20', experiments, outcomes)


# CODE JESSE

In [ ]:
n_scenarios = 50000
np.random.seed(21)
policy = policies_21[0]

# Load your model and define parameters
dike_model, planning_steps = get_model_for_problem_formulation(3)

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(
        scenarios=n_scenarios,
        policies=policy,
        uncertainty_sampling=Samplers.LHS
    )

experiments, outcomes = results
save_experiments_and_outcomes('policy_7_s21', experiments, outcomes)


# CODE PIEN

In [ ]:
n_scenarios = 50000
np.random.seed(21)
policy = policies_21[1]

# Load your model and define parameters
dike_model, planning_steps = get_model_for_problem_formulation(3)

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(
        scenarios=n_scenarios,
        policies=policy,
        uncertainty_sampling=Samplers.LHS
    )

experiments, outcomes = results
save_experiments_and_outcomes('policy_8_s21', experiments, outcomes)


In [ ]:
flattened_data = {key: value.flatten() for key, value in outcomes.items()}
df_outcomes = pd.DataFrame(flattened_data)
df_outcomes.columns